In [1]:
import pandas as pd
import datetime as dt
from modules import crfb
from modules import covid

In [2]:
# Get the most recent dataset that we have on file
crfb_df = crfb.get_data_single()
# Download all the datasets we have, using the column 'Date CRFB Downloaded'
#crfb_df = crfb.get_data_all()
crfb_df

loading data from date 2021-02-09 : file 20210209_cmt.xlsx...


,Recipient State,Amount Committed/Disbursed,Date,Recipient Type,Legislation,Agency
0,Alabama,4.243599e+09,2020-04-15,Household,CARES Act,IRS
1,Alaska,6.098270e+08,2020-04-15,Household,CARES Act,IRS
2,Arizona,5.918266e+09,2020-04-15,Household,CARES Act,IRS
3,Arkansas,2.650191e+09,2020-04-15,Household,CARES Act,IRS
4,California,2.960029e+10,2020-04-15,Household,CARES Act,IRS
...,...,...,...,...,...,...
7303,California,3.325000e+07,2020-12-31,Small Business,CARES Act,Federal Reserve
7304,Oregon,1.425000e+07,2020-12-31,Small Business,CARES Act,Federal Reserve
7305,California,1.900000e+06,2020-12-31,Small Business,CARES Act,Federal Reserve
7306,Virginia,1.779091e+07,2020-12-31,Small Business,CARES Act,Federal Reserve


In [3]:
# To filter by date use
#covid_df = covid.get_data(date_min=dt.datetime(2020, 10, 23), date_max=dt.datetime(2021, 2, 9))
# else, use this
covid_df = covid.get_data()
covid_df

,date,state,positive,negative,onVentilatorCumulative,recovered,death,hospitalized,positiveTests
0,2021-02-09,Alaska,53809.0,0.0,0.0,0.0,280.0,1223.0,65100
1,2021-02-09,Alabama,474666.0,1825185.0,1463.0,252880.0,8579.0,43499.0,0
2,2021-02-09,Arkansas,308848.0,2297201.0,1464.0,288774.0,5148.0,14174.0,74670
3,2021-02-09,Arizona,787268.0,2834530.0,0.0,108965.0,14286.0,54967.0,0
4,2021-02-09,California,3354591.0,0.0,0.0,0.0,44477.0,0.0,0
...,...,...,...,...,...,...,...,...,...
17315,2020-01-17,Washington,0.0,0.0,0.0,0.0,0.0,0.0,0
17316,2020-01-16,Washington,0.0,0.0,0.0,0.0,0.0,0.0,0
17317,2020-01-15,Washington,0.0,0.0,0.0,0.0,0.0,0.0,0
17318,2020-01-14,Washington,0.0,0.0,0.0,0.0,0.0,0.0,0


In [40]:
state_leg = crfb_df.groupby('Recipient State')['Legislation'].count().sort_values(ascending=False).to_frame()
state_leg.reset_index(inplace=True)

In [41]:
state_money = crfb_df.groupby('Recipient State')['Amount Committed/Disbursed'].sum().sort_values(ascending=False).to_frame()
state_money.reset_index(inplace=True)
state_money['Amount Committed/Disbursed'] = state_money.apply(lambda x: "${0:,.2f}".format(x['Amount Committed/Disbursed']), axis=1)
state_money.rename(columns={"Recipient State": "state"}, inplace=True)

In [42]:
state_death = covid_df.groupby('state')['death'].max().to_frame()
state_death.reset_index(inplace=True)

In [43]:
state_cases = covid_df.groupby('state')['positive'].max().to_frame()
state_cases.reset_index(inplace=True)

In [46]:
sample_state = pd.merge(state_cases,state_death,on='state')
sample_state['Death %'] = round(sample_state['death']/sample_state['positive'] * 100,2)

In [56]:
master_frame = pd.merge(sample_state,state_money,on='state',how='left').sort_values(by='Death %',ascending=False,axis=0).reset_index(drop=True)
master_frame

,state,positive,death,Death %,Amount Committed/Disbursed
0,New Jersey,728304.0,22103.0,3.03,"$55,065,781,573.00"
1,Massachusetts,546896.0,15124.0,2.77,"$45,919,622,915.00"
2,Connecticut,264608.0,7298.0,2.76,"$18,889,945,792.00"
3,Pennsylvania,876913.0,22620.0,2.58,"$72,126,939,806.00"
4,Michigan,623731.0,15925.0,2.55,"$53,167,573,494.00"
5,New York,1487086.0,36481.0,2.45,"$153,340,195,800.00"
6,Mississippi,282969.0,6342.0,2.24,"$12,296,763,111.00"
7,Louisiana,414354.0,9162.0,2.21,"$22,276,314,564.00"
8,Maryland,365529.0,7413.0,2.03,"$30,594,832,680.00"
9,New Mexico,178280.0,3430.0,1.92,"$8,003,573,514.00"


In [10]:
## day_change
# day change per state per date. date, year, month, day, +positive +death +recovered +money recieved
## week_change
# week change per state, same values
## month_change
# month change per state, same values